# Capítulo 03: Configuração de Credenciais AWS/S3

Este capítulo demonstra diferentes métodos para configurar credenciais de acesso ao S3 no DuckDB, incluindo Secrets explícitos, Cadeia de Credenciais (Variáveis de Ambiente) e Perfis AWS.

In [1]:
import duckdb
import os
import boto3
from botocore.exceptions import ClientError

# ==============================================================================
# SETUP MINIO
# ==============================================================================
print(f"--- Iniciando Capítulo 03: Configuração de Credenciais ---")

MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
BUCKET_NAME = "learn-duckdb-s3"

s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)
try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
except ClientError:
    pass

# Helper to write verify file
with open("creds.csv", "w") as f:
    f.write("id,msg\n1,success")
s3_client.upload_file("creds.csv", BUCKET_NAME, "creds.csv")
os.remove("creds.csv")
print("Ambiente configurado.")

--- Iniciando Capítulo 03: Configuração de Credenciais ---
Ambiente configurado.


## 🦆 Setup DuckDB

In [2]:
con = duckdb.connect(database=':memory:')
con.execute("INSTALL httpfs; LOAD httpfs;")

## 🔑 Método 1: CREATE SECRET (Explícito)

A forma mais direta de configurar credenciais é criar um SECRET do tipo S3 com todas as informações necessárias.

In [3]:
print("\n--- Método 1: CREATE SECRET (Explícito) ---")
con.execute(f"""
CREATE OR REPLACE SECRET secret_explicit (
    TYPE S3,
    KEY_ID '{MINIO_ACCESS_KEY}',
    SECRET '{MINIO_SECRET_KEY}',
    REGION 'us-east-1',
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")
res = con.execute(f"SELECT * FROM 's3://{BUCKET_NAME}/creds.csv'").fetchall()
print(f"Leitura com Secret Explícito: {res}")


--- Método 1: CREATE SECRET (Explícito) ---
Leitura com Secret Explícito: [(1, 'success')]


## 🔗 Método 2: Variáveis de Ambiente + Credential Chain

O DuckDB suporta o AWS Credential Chain padrão. Se as variáveis de ambiente `AWS_ACCESS_KEY_ID` e `AWS_SECRET_ACCESS_KEY` estiverem definidas, podemos usar `PROVIDER CREDENTIAL_CHAIN`.

**Nota:** Para MinIO e outros S3-compatíveis, ainda precisamos especificar o `ENDPOINT` manualmente no secret, pois as variáveis de ambiente padrão da AWS não cobrem endpoints customizados de forma universal.

In [4]:
print("\n--- Método 2: Variáveis de Ambiente + Credential Chain ---")
# Limpar secrets anteriores para garantir que estamos testando o novo método
con.execute("DROP SECRET IF EXISTS secret_explicit")

# Definir Env Vars
os.environ["AWS_ACCESS_KEY_ID"] = MINIO_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = MINIO_SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Criar secreto usando PROVIDER CREDENTIAL_CHAIN
con.execute(f"""
CREATE OR REPLACE SECRET secret_from_env (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")

res = con.execute(f"SELECT * FROM 's3://{BUCKET_NAME}/creds.csv'").fetchall()
print(f"Leitura com Env Vars: {res}")


--- Método 2: Variáveis de Ambiente + Credential Chain ---
Leitura com Env Vars: [(1, 'success')]


## 👤 Método 3: AWS Profile (Config)

Também é possivel usar `PROVIDER CONFIG` para ler do arquivo `~/.aws/config` e `~/.aws/credentials` especificando um profile.

In [5]:
print("\n--- Método 3: Profile (Simulado via boto3 config logic) ---")
# O provider 'config' tentaria ler ~/.aws/config. Não vamos modificar o home do usuário aqui.
# Mas a sintaxe seria:
print("Exemplo de sintaxe (não executado):")
print("""
CREATE SECRET secret_profile (
    TYPE S3,
    PROVIDER CONFIG,
    PROFILE 'my-minio-profile'
);
""")


--- Método 3: Profile (Simulado via boto3 config logic) ---
Exemplo de sintaxe (não executado):

CREATE SECRET secret_profile (
    TYPE S3,
    PROVIDER CONFIG,
    PROFILE 'my-minio-profile'
);



# Capitulo 03 Configuracao Credenciais Aws

Notebook gerado automaticamente a partir do código fonte python.


In [6]:
import duckdb
import os
import boto3
from botocore.exceptions import ClientError

# ==============================================================================
# SETUP MINIO
# ==============================================================================
print(f"--- Iniciando Capítulo 03: Configuração de Credenciais ---")

MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
BUCKET_NAME = "learn-duckdb-s3"

s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)
try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
    print(f"Bucket '{BUCKET_NAME}' verificado/criado.")
except ClientError:
    print(f"Bucket '{BUCKET_NAME}' já existe.")

# Helper to write verify file
with open("creds.csv", "w") as f:
    f.write("id,msg\n1,success")
s3_client.upload_file("creds.csv", BUCKET_NAME, "creds.csv")
os.remove("creds.csv")
print("Arquivo de teste 'creds.csv' enviado.")

--- Iniciando Capítulo 03: Configuração de Credenciais ---
Bucket 'learn-duckdb-s3' já existe.
Arquivo de teste 'creds.csv' enviado.


In [7]:
con = duckdb.connect(database=':memory:')
con.execute("INSTALL httpfs; LOAD httpfs;")

print("\n--- Método 1: CREATE SECRET (Explícito) ---")
con.execute(f"""
CREATE OR REPLACE SECRET secret_explicit (
    TYPE S3,
    KEY_ID '{MINIO_ACCESS_KEY}',
    SECRET '{MINIO_SECRET_KEY}',
    REGION 'us-east-1',
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")
res = con.execute(f"SELECT * FROM 's3://{BUCKET_NAME}/creds.csv'").fetchall()
print(f"Leitura com Secret Explícito: {res}")


--- Método 1: CREATE SECRET (Explícito) ---
Leitura com Secret Explícito: [(1, 'success')]


In [8]:
print("\n--- Método 2: Variáveis de Ambiente + Credential Chain ---")
# Limpar secrets
con.execute("DROP SECRET IF EXISTS secret_explicit")
# Definir Env Vars
os.environ["AWS_ACCESS_KEY_ID"] = MINIO_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = MINIO_SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Nota: Para MinIO local sem SSL, ainda precisamos configurar o Endpoint explicitamente no Secret,
# pois as vars de ambiente AWS padrão não cobrem ENDPOINT customizado para o DuckDB automaticamente
# sem usar configurações específicas OU um secret que use provider chain mas sobrescreva endpoint.
con.execute(f"""
CREATE OR REPLACE SECRET secret_from_env (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")

res = con.execute(f"SELECT * FROM 's3://{BUCKET_NAME}/creds.csv'").fetchall()
print(f"Leitura com Env Vars: {res}")


--- Método 2: Variáveis de Ambiente + Credential Chain ---
Leitura com Env Vars: [(1, 'success')]
